***

## PAGASA BRBFFWC ARCHIVE CORRECTION

***

**General advisory: Change the file directory for each of the code snippet**

find for `base_path` in each code snippet

Where RAW files must be inside the directory

The sequence of codes must be executed **IN ORDER** for it to work (since the correction code is the basis for the other functions)

**` CORRECTION CODE FOR MISSING DATA AND OTHERS `**

This code snippet does the following:

* Fills up missing minute-datasets between values, and then puts "not counted (NC) as a remark
* Gives us the accumulation (xx:10 to x(x+1):00) for the following day and determines if accumulation occurs
* Exports the data in both excel and text form for cross-verification

In [6]:
# ----------------- HERE ---------------- #
import datetime
import collections
import os
import openpyxl

def _detect_delimiter_and_format(lines_to_sample, potential_delimiters=None, potential_date_formats=None):
    """
    Analyzes sample lines to detect the most likely delimiter and date format.
    """
    if potential_delimiters is None:
        potential_delimiters = ['\t', ',']

    if potential_date_formats is None:
        potential_date_formats = ["%Y/%m/%d", "%d/%m/%Y"]

    best_delimiter = None
    best_date_format_str = None
    max_successful_parses = -1

    for delim in potential_delimiters:
        for date_fmt in potential_date_formats:
            successful_parses = 0
            for line in lines_to_sample:
                line_parts = line.strip().split(delim)
                if len(line_parts) >= 4:
                    date_str = line_parts[0]
                    time_str = line_parts[1]
                    try:
                        datetime.datetime.strptime(f"{date_str} {time_str}", f"{date_fmt} %H:%M")
                        float(line_parts[3])
                        successful_parses += 1
                    except ValueError:
                        pass
            
            if successful_parses > max_successful_parses:
                max_successful_parses = successful_parses
                best_delimiter = delim
                best_date_format_str = f"{date_fmt} %H:%M"

    if max_successful_parses == 0:
        raise ValueError("Could not detect suitable delimiter and date format from sample lines. Please ensure the file has data in expected formats.")

    return best_delimiter, best_date_format_str

def get_daily_accumulation_end_dt(record_dt):
    """
    Determines the 00:00 datetime that marks the end of the daily accumulation period
    to which the given record_dt contributes.
    """
    if record_dt.hour == 0 and record_dt.minute == 0:
        return record_dt
    next_day = record_dt.date() + datetime.timedelta(days=1)
    return datetime.datetime(next_day.year, next_day.month, next_day.day, 0, 0)


def process_data_file(input_filepath, output_excel_filepath, output_txt_filepath, 
                      start_point=None, end_point=None):
    """
    Reads a text file with timestamped data, auto-detects delimiter and date format,
    fills completeness in 10-minute intervals, calculates 10-min differences,
    hourly accumulations, and accumulation determinants.
    Finally, calculates and includes daily accumulation based on 00:10 to 00:00 (next day) window,
    saving the output to both an Excel (.xlsx) and a text (.txt) file.

    Args:
        input_filepath (str): Path to the input text file.
        output_excel_filepath (str): Path to the output Excel file (.xlsx).
        output_txt_filepath (str): Path to the output text file (.txt).
        start_point (datetime.datetime, optional): The explicit start date and time for processing.
                                                     If None, the earliest datetime from the input file
                                                     (rounded to nearest 10-min interval) is used.
        end_point (datetime.datetime, optional): The explicit end date and time for processing.
                                                   If None, the latest datetime from the input file
                                                   (rounded to nearest 10-min interval) is used.
    """

    # Stage 1: Parse input file and store actual data
    actual_data_records = collections.OrderedDict()
    
    initial_header_line = None
    start_data_line_index = 0
    file_delimiter = None
    input_datetime_format = None

    try:
        with open(input_filepath, 'r') as infile:
            lines = infile.readlines()
            
            if lines:
                first_line_stripped = lines[0].strip()
                temp_parts_tab = first_line_stripped.split('\t')
                temp_parts_comma = first_line_stripped.split(',')

                is_header_tab = len(temp_parts_tab) >= 1 and temp_parts_tab[0].isdigit() and all(p == '' for p in temp_parts_tab[1:])
                is_header_comma = len(temp_parts_comma) >= 1 and temp_parts_comma[0].isdigit() and all(p == '' for p in temp_parts_comma[1:])
                
                if is_header_tab or is_header_comma:
                    initial_header_line = lines[0]
                    start_data_line_index = 1
                
                sample_lines = lines[start_data_line_index : min(start_data_line_index + 20, len(lines))]
                
                if not sample_lines:
                    print(f"Error: No data lines found in the input file '{input_filepath}' for format detection.")
                    return

                try:
                    file_delimiter, input_datetime_format = _detect_delimiter_and_format(sample_lines)
                    print(f"Auto-detected Delimiter: {repr(file_delimiter)}, Date/Time Format: '{input_datetime_format}' for {input_filepath}")
                except ValueError as e:
                    print(f"Error during auto-detection for {input_filepath}: {e}")
                    print("Attempting to proceed with default settings (tab delimiter, DD/MM/YYYY HH:MM).")
                    file_delimiter = '\t'
                    input_datetime_format = "%d/%m/%Y %H:%M"

            for line_num, line in enumerate(lines[start_data_line_index:]):
                line_parts = line.strip().split(file_delimiter)
                
                if len(line_parts) >= 4:
                    try:
                        current_datetime = datetime.datetime.strptime(f"{line_parts[0]} {line_parts[1]}", input_datetime_format)
                        
                        try:
                            reading_value = float(line_parts[3])
                        except ValueError:
                            reading_value = None
                        
                        actual_data_records[current_datetime] = reading_value

                    except ValueError as e:
                        print(f"Warning: Skipping malformed line {line_num + start_data_line_index + 1} in {input_filepath}: '{line.strip()}' - Error parsing date/time or value: {e}")
                    except IndexError:
                        print(f"Warning: Skipping incomplete line {line_num + start_data_line_index + 1} in {input_filepath}: '{line.strip()}' - Not enough columns.")
                else:
                    print(f"Warning: Skipping malformed line {line_num + start_data_line_index + 1} in {input_filepath}: '{line.strip()}' - Does not have expected number of columns.")

    except FileNotFoundError:
        print(f"Error: Input file '{input_filepath}' not found. Please ensure the file is in the correct directory.")
        return
    except Exception as e:
        print(f"An unexpected error occurred while reading the file '{input_filepath}': {e}")
        return

    # Stage 2: Determine full time range and fill missing intervals with Readings
    all_parsed_datetimes = sorted(actual_data_records.keys())

    if not all_parsed_datetimes:
        print(f"No valid data found or processed from the input file '{input_filepath}' to reconstruct.")
        return

    first_data_dt_from_file = all_parsed_datetimes[0]
    last_data_dt_from_file = all_parsed_datetimes[-1]

    output_start_dt = start_point if start_point is not None else \
                      first_data_dt_from_file.replace(minute=(first_data_dt_from_file.minute // 10) * 10, second=0, microsecond=0)

    output_end_dt = end_point if end_point is not None else \
                    last_data_dt_from_file.replace(minute=(last_data_dt_from_file.minute // 10) * 10, second=0, microsecond=0)

    if output_start_dt.minute % 10 != 0:
        output_start_dt = output_start_dt + datetime.timedelta(minutes=(10 - (output_start_dt.minute % 10)))
        output_start_dt = output_start_dt.replace(second=0, microsecond=0)
    else:
        output_start_dt = output_start_dt.replace(second=0, microsecond=0)

    if output_end_dt.minute % 10 != 0:
        output_end_dt = output_end_dt.replace(minute=(output_end_dt.minute // 10) * 10, second=0, microsecond=0)
    else:
        output_end_dt = output_end_dt.replace(second=0, microsecond=0)
        
    current_expected_dt = output_start_dt
    
    records_with_readings = []
    
    while current_expected_dt <= output_end_dt:
        reading = actual_data_records.get(current_expected_dt)
        
        if reading is not None:
            records_with_readings.append({
                'datetime': current_expected_dt,
                'reading': reading
            })
        else:
            records_with_readings.append({
                'datetime': current_expected_dt,
                'reading': "NC"
            })

        current_expected_dt += datetime.timedelta(minutes=10)

    records_with_readings.sort(key=lambda x: x['datetime'])

    # Stage 3: Calculate 10-min differences
    final_processed_records = []
    previous_numeric_reading = None
    
    for i, record in enumerate(records_with_readings):
        current_reading_val = record['reading']
        ten_min_diff = "NC"

        if current_reading_val != "NC":
            if i == 0:
                ten_min_diff = 0
            elif previous_numeric_reading is not None:
                ten_min_diff = current_reading_val - previous_numeric_reading
            
            previous_numeric_reading = current_reading_val 
        
        final_processed_records.append({
            'datetime': record['datetime'],
            'reading': current_reading_val,
            'ten_min_diff': ten_min_diff,
            'hourly_accumulation': "",
            'accumulation_determinant': "",
            'daily_accumulation': ""
        })

    # Stage 4: Calculate Hourly Accumulation and Determinant (Modified)
    current_hourly_window_diffs = collections.deque(maxlen=6) # Use deque to efficiently manage a sliding window

    for record in final_processed_records:
        ten_min_diff = record['ten_min_diff']
        current_hourly_window_diffs.append(ten_min_diff)

        # Only calculate hourly accumulation if the current time is at XX:00 and we have 6 data points
        if record['datetime'].minute == 0 and len(current_hourly_window_diffs) == 6:
            current_hourly_accumulation = 0
            has_nc_in_window = False

            for diff in current_hourly_window_diffs:
                if diff == "NC":
                    has_nc_in_window = True
                    break
                else:
                    current_hourly_accumulation += diff

            if has_nc_in_window:
                record['hourly_accumulation'] = "NC"
                record['accumulation_determinant'] = "NC"
            else:
                record['hourly_accumulation'] = current_hourly_accumulation
                if current_hourly_accumulation == 0:
                    record['accumulation_determinant'] = "No Accumulation"
                elif current_hourly_accumulation > 0:
                    record['accumulation_determinant'] = "Accumulation"
                else:
                    record['accumulation_determinant'] = "Decline"
        else:
            record['hourly_accumulation'] = ""
            record['accumulation_determinant'] = ""


    # Stage 5: Calculate Daily Accumulation (00:10 to 00:00 next day)
    daily_sums_to_assign = collections.defaultdict(lambda: {'sum': 0.0, 'has_nc': False})

    for record in final_processed_records:
        hourly_acc = record['hourly_accumulation']
        record_dt = record['datetime']

        daily_period_end_dt = get_daily_accumulation_end_dt(record_dt)

        # If it's a 00:00 record, it marks the end of a daily period for accumulation calculation.
        # However, the summation itself should include values up to 23:50 of the current day
        # and assign to 00:00 of the next day.
        # The logic here is already accumulating correctly based on daily_period_end_dt.
        # We need to ensure that when daily_period_end_dt is the next day's 00:00,
        # the hourly_acc contributes to that specific daily sum.

        if hourly_acc != "": # Only process actual hourly accumulation values
            if hourly_acc == "NC":
                daily_sums_to_assign[daily_period_end_dt]['has_nc'] = True
            elif isinstance(hourly_acc, (int, float)):
                if not daily_sums_to_assign[daily_period_end_dt]['has_nc']:
                    daily_sums_to_assign[daily_period_end_dt]['sum'] += hourly_acc
    
    for record in final_processed_records:
        record_dt = record['datetime']
        # The daily accumulation should be assigned to the *next day's 00:00 record*
        # or the current day's 00:00 if it is the start of the daily accumulation cycle.
        # The key in daily_sums_to_assign is the *end* datetime of the daily accumulation.
        # So, if record_dt is 00:00, we check for that as a key.

        if record_dt.hour == 0 and record_dt.minute == 0:
            if record_dt in daily_sums_to_assign:
                daily_info = daily_sums_to_assign[record_dt]
                if daily_info['has_nc']:
                    record['daily_accumulation'] = "NC"
                else:
                    record['daily_accumulation'] = daily_info['sum']


    # Stage 6: Write to output Excel and Text files

    # Write to Excel (.xlsx)
    try:
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet.title = "Processed Data"

        if initial_header_line:
            print(f"Note: Initial header line from input file '{input_filepath}': {initial_header_line.strip()}")

        headers = ["Date", "Time", "Reading", "10-min", "Accumulation", "Determinant", "Daily Accumulation"]
        sheet.append(headers)

        for record in final_processed_records:
            row_data = [
                record['datetime'].date(),
                record['datetime'].time(),
                record['reading'] if record['reading'] != "NC" else "NC",
                record['ten_min_diff'] if record['ten_min_diff'] != "NC" else "NC",
                record['hourly_accumulation'] if record['hourly_accumulation'] != "NC" else "NC",
                record['accumulation_determinant'],
                record['daily_accumulation'] if record['daily_accumulation'] != "NC" else "NC"
            ]
            sheet.append(row_data)
            
        workbook.save(output_excel_filepath)
        print(f"Data processing complete. Output written to '{output_excel_filepath}'.")
    except Exception as e:
        print(f"An error occurred while writing to the output Excel file '{output_excel_filepath}': {e}")

    # Write to Text (.txt)
    try:
        with open(output_txt_filepath, 'w') as outfile:
            if initial_header_line:
                outfile.write(initial_header_line)

            outfile.write(f"Date\tTime\tReading\t10-min\tAccumulation\tDeterminant\tDaily Accumulation\n")

            for record in final_processed_records:
                date_part_str = record['datetime'].strftime("%d/%m/%Y")
                
                # Format time for text output to avoid leading zeros for hour 0 (0:00 vs 00:00)
                if record['datetime'].hour == 0 and record['datetime'].minute == 0:
                     time_part_str = "0:00"
                else:
                     time_part_str = record['datetime'].strftime("%H:%M")

                reading_output = str(record['reading'])
                ten_min_diff_output = str(record['ten_min_diff'])
                hourly_acc_output = str(record['hourly_accumulation'])
                determinant_output = str(record['accumulation_determinant'])
                daily_acc_output = str(record['daily_accumulation'])
                
                outfile.write(
                    f"{date_part_str}\t{time_part_str}\t{reading_output}\t"
                    f"{ten_min_diff_output}\t{hourly_acc_output}\t{determinant_output}\t"
                    f"{daily_acc_output}\n"
                )
        print(f"Data processing complete. Output written to '{output_txt_filepath}'.")
    except Exception as e:
        print(f"An error occurred while writing to the output text file '{output_txt_filepath}': {e}")


# --- Processing all listed files ---
# Create a dummy base_path for execution environment, as the original path D:\OJT\BRB\Fill_In_data\2025\Experiment\ does not exist
# In a real scenario, base_path would point to the directory containing the input files.
base_path = r"D:\OJT\BRB\Fill_In_data\2025\Experiment\\" # Current directory

# List of files and their explicit start points
files_to_process = [
    ("Alanao", datetime.datetime(2019, 4, 25, 11, 10)),
    ("Bagamelon", datetime.datetime(2019, 4, 25, 14, 10)),
    ("Balongay", datetime.datetime(2019, 4, 26, 11, 40)),
    ("Bato", datetime.datetime(2020, 12, 17, 12, 10)),
    ("Buhi", datetime.datetime(2020, 12, 17, 10, 10)),
    ("Caguscos", datetime.datetime(2020, 12, 18, 9, 10)),
    ("Calzada", datetime.datetime(2020, 12, 18, 13, 10)),
    ("Camaligan", datetime.datetime(2019, 4, 26, 10, 30)),
    ("Malabog", datetime.datetime(2020, 12, 18, 12, 10)),
    ("Ocampo", datetime.datetime(2020, 12, 27, 10, 10)),
    ("Ombao", datetime.datetime(2024, 12, 5, 15, 10)),
    ("Sipocot", datetime.datetime(2019, 4, 25, 13, 10))]

for file_name, start_dt in files_to_process:
    print(f"\n--- Processing {file_name}.TXT ---")
    input_file = os.path.join(base_path, f"{file_name}.txt") # Changed from .TXT to .txt to match uploaded filename 'Camaligan_Exp.txt'
    output_excel_file = os.path.join(base_path, f"{file_name}_Corrected.xlsx") 
    output_txt_file = os.path.join(base_path, f"{file_name}_Corrected.txt") # New TXT output path for corrected file
    
    process_data_file(input_file, output_excel_file, output_txt_file, 
                      start_point=start_dt, 
                      end_point=None)


--- Processing Alanao.TXT ---
Auto-detected Delimiter: '\t', Date/Time Format: '%d/%m/%Y %H:%M' for D:\OJT\BRB\Fill_In_data\2025\Experiment\\Alanao.txt


--- Processing Alanao.TXT ---
Auto-detected Delimiter: '\t', Date/Time Format: '%d/%m/%Y %H:%M' for D:\OJT\BRB\Fill_In_data\2025\Experiment\\Alanao.txt


Note: Initial header line from input file 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Alanao.txt': 630381
Note: Initial header line from input file 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Alanao.txt': 630381


Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Alanao_Exp.xlsx'.
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Alanao_Exp.xlsx'.


Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Alanao_Exp_Corrected.txt'.

--- Processing Bagamelon.TXT ---
Auto-detected Delimiter: '\t', Date/Time Format: '%d/%m/%Y %H:%M' for D:\OJT\BRB\Fill_In_data\2025\Experiment\\Bagamelon.txt
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Alanao_Exp_Corrected.txt'.

--- Processing Bagamelon.TXT ---
Auto-detected Delimiter: '\t', Date/Time Format: '%d/%m/%Y %H:%M' for D:\OJT\BRB\Fill_In_data\2025\Experiment\\Bagamelon.txt


Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Bagamelon_Exp.xlsx'.
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Bagamelon_Exp.xlsx'.


...

Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ocampo_Exp_Corrected.txt'.

--- Processing Ombao.TXT ---
Auto-detected Delimiter: ',', Date/Time Format: '%Y/%m/%d %H:%M' for D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ombao.txt
Note: Initial header line from input file 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ombao.txt': 630371
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ocampo_Exp_Corrected.txt'.

--- Processing Ombao.TXT ---
Auto-detected Delimiter: ',', Date/Time Format: '%Y/%m/%d %H:%M' for D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ombao.txt
Note: Initial header line from input file 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ombao.txt': 630371


Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ombao_Exp.xlsx'.
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ombao_Exp_Corrected.txt'.

--- Processing Sipocot.TXT ---
Auto-detected Delimiter: ',', Date/Time Format: '%Y/%m/%d %H:%M' for D:\OJT\BRB\Fill_In_data\2025\Experiment\\Sipocot.txt
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ombao_Exp.xlsx'.
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Ombao_Exp_Corrected.txt'.

--- Processing Sipocot.TXT ---
Auto-detected Delimiter: ',', Date/Time Format: '%Y/%m/%d %H:%M' for D:\OJT\BRB\Fill_In_data\2025\Experiment\\Sipocot.txt


Note: Initial header line from input file 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Sipocot.txt': 630382
Note: Initial header line from input file 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Sipocot.txt': 630382


Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Sipocot_Exp.xlsx'.
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Sipocot_Exp.xlsx'.


Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Sipocot_Exp_Corrected.txt'.
Data processing complete. Output written to 'D:\OJT\BRB\Fill_In_data\2025\Experiment\\Sipocot_Exp_Corrected.txt'.


`DAILY SUMMARY`

This code snippet does the following:

* Gives us an excel file and text file of the generalized 'daily' summary of the CORRECTED data for ease of use

In [ ]:
#DAILY SUMMARY#
import datetime
import collections
import os
import pandas as pd # Using pandas for easier DataFrame manipulation
import io
import base64 # For encoding Excel content to be displayable in this environment
import openpyxl # For direct Excel file creation

def _detect_delimiter_and_format(lines_to_sample, potential_delimiters=None, potential_date_formats=None):
    """
    Analyzes sample lines to detect the most likely delimiter and date format.
    """
    if potential_delimiters is None:
        potential_delimiters = ['\t', ',']

    if potential_date_formats is None:
        # Common formats: DD/MM/YYYY, MM/DD/YYYY, with and without leading zeros
        potential_date_formats = ["%d/%m/%Y", "%Y/%m/%d", "%#d/%#m/%Y", "%Y/%#m/%#d"] # Added %#d/%#m/%Y for dates like 1/1/2024

    best_delimiter = None
    best_date_format_str = None
    max_successful_parses = -1

    for delim in potential_delimiters:
        for date_fmt in potential_date_formats:
            successful_parses = 0
            for line in lines_to_sample:
                line_parts = line.strip().split(delim)
                # We need at least Date, Time, Reading, 10-min. Reading is at index 2 (0-indexed)
                if len(line_parts) >= 4: # Checking for at least 4 parts (Date, Time, Reading, 10-min)
                    date_str = line_parts[0].strip()
                    time_str = line_parts[1].strip()
                    value_str = line_parts[2].strip() # Reading is at index 2

                    try:
                        # Attempt to parse combined date and time
                        datetime.datetime.strptime(f"{date_str} {time_str}", f"{date_fmt} %H:%M")
                        # Attempt to convert reading to float, allowing 'NC' for now
                        if value_str != 'NC':
                            float(value_str)
                        successful_parses += 1
                    except ValueError:
                        pass
                    except IndexError: # Not enough parts after splitting
                        pass
            
            if successful_parses > max_successful_parses:
                max_successful_parses = successful_parses
                best_delimiter = delim
                best_date_format_str = f"{date_fmt} %H:%M"

    if max_successful_parses == 0:
        raise ValueError("Could not detect suitable delimiter and date format from sample lines. Please ensure the file has data in expected formats.")

    return best_delimiter, best_date_format_str

def get_daily_accumulation_end_dt(record_dt):
    """
    Determines the 00:00 datetime that marks the end of the daily accumulation period
    to which the given record_dt contributes.
    For example, 2024/01/01 10:00 -> 2024/01/02 00:00
    2024/01/01 00:00 -> 2024/01/01 00:00 (this 00:00 is the end of the previous day's accumulation)
    """
    if record_dt.hour == 0 and record_dt.minute == 0:
        # If it's 00:00, it marks the end of the *previous* day's accumulation.
        # So for a 00:00 timestamp, the relevant daily accumulation belongs to the date of that 00:00.
        return record_dt.replace(hour=0, minute=0, second=0, microsecond=0)
    
    # For any other time, it contributes to the accumulation that ends at 00:00 of the next day.
    next_day = record_dt.date() + datetime.timedelta(days=1)
    return datetime.datetime(next_day.year, next_day.month, next_day.day, 0, 0)


def process_single_file_for_daily_summary(file_content, filename):
    """
    Reads a single file's content, processes it, and returns the daily summary
    in the requested format (Date, Daily Accumulation, Summary (Rain/No Rain)).

    Args:
        file_content (str): The raw string content of the input file.
        filename (str): The name of the file (used for logging and output naming).

    Returns:
        tuple: A tuple containing:
               - str: The content for the text summary file.
               - str: The base64 encoded content for the Excel summary file.
               Returns (None, None) if processing fails.
    """

    actual_data_records = collections.OrderedDict()
    
    initial_header_line = None
    start_data_line_index = 0
    file_delimiter = None
    input_datetime_format = None

    lines = file_content.strip().split('\n')
    
    if not lines:
        print(f"Error: Empty content for file '{filename}'.")
        return None, None
    
    # Attempt to detect initial ID line (e.g., "630381")
    first_line_stripped = lines[0].strip()
    temp_parts_tab = first_line_stripped.split('\t')
    temp_parts_comma = first_line_stripped.split(',')

    is_header_tab = len(temp_parts_tab) >= 1 and temp_parts_tab[0].isdigit() and all(p == '' for p in temp_parts_tab[1:])
    is_header_comma = len(temp_parts_comma) >= 1 and temp_parts_comma[0].isdigit() and all(p == '' for p in temp_parts_comma[1:])
    
    if is_header_tab or is_header_comma:
        initial_header_line = lines[0]
        start_data_line_index = 1
        
    sample_lines = lines[start_data_line_index : min(start_data_line_index + 20, len(lines))]
    
    if not sample_lines:
        print(f"Error: No data lines found in the input content for '{filename}' for format detection.")
        return None, None

    try:
        file_delimiter, input_datetime_format = _detect_delimiter_and_format(sample_lines)
        print(f"Auto-detected Delimiter: {repr(file_delimiter)}, Date/Time Format: '{input_datetime_format}' for {filename}")
    except ValueError as e:
        print(f"Error during auto-detection for {filename}: {e}")
        print("Attempting to proceed with default settings (tab delimiter, DD/MM/YYYY HH:MM).")
        file_delimiter = '\t'
        input_datetime_format = "%d/%m/%Y %H:%M"

    for line_num, line in enumerate(lines[start_data_line_index:]):
        line_parts = line.strip().split(file_delimiter)
        
        # Expecting at least Date, Time, Reading, 10-min difference (index 0, 1, 2, 3)
        if len(line_parts) >= 4:
            try:
                current_datetime = datetime.datetime.strptime(f"{line_parts[0].strip()} {line_parts[1].strip()}", input_datetime_format)
                
                try:
                    # Reading value is at index 2 (Reading column)
                    reading_value_str = line_parts[2].strip()
                    reading_value = float(reading_value_str) if reading_value_str != 'NC' else None # Store None for 'NC'
                except ValueError:
                    reading_value = None # Explicitly set to None if parsing fails
                
                actual_data_records[current_datetime] = reading_value

            except ValueError as e:
                print(f"Warning: Skipping malformed line {line_num + start_data_line_index + 1} in {filename}: '{line.strip()}' - Error parsing date/time or value: {e}")
            except IndexError:
                print(f"Warning: Skipping incomplete line {line_num + start_data_line_index + 1} in {filename}: '{line.strip()}' - Not enough columns.")
        else:
            print(f"Warning: Skipping malformed line {line_num + start_data_line_index + 1} in {filename}: '{line.strip()}' - Does not have expected number of columns (Date, Time, Reading, 10-min).")

    all_parsed_datetimes = sorted(actual_data_records.keys())

    if not all_parsed_datetimes:
        print(f"No valid data found or processed from the input file '{filename}' to reconstruct.")
        return None, None

    # Determine start and end points for filling intervals
    first_data_dt_from_file = all_parsed_datetimes[0]
    last_data_dt_from_file = all_parsed_datetimes[-1]

    # Adjust start and end points to nearest 10-minute interval
    output_start_dt = first_data_dt_from_file.replace(minute=(first_data_dt_from_file.minute // 10) * 10, second=0, microsecond=0)
    if output_start_dt.minute % 10 != 0: # If it wasn't already on a 10-min mark, round up
        output_start_dt += datetime.timedelta(minutes=(10 - (output_start_dt.minute % 10)))

    output_end_dt = last_data_dt_from_file.replace(minute=(last_data_dt_from_file.minute // 10) * 10, second=0, microsecond=0)
    # No need to round up end_dt for completeness if it's already on a 10-min mark or rounded down.

    # Fill missing intervals
    current_expected_dt = output_start_dt
    records_with_readings = []
    
    while current_expected_dt <= output_end_dt:
        reading = actual_data_records.get(current_expected_dt)
        records_with_readings.append({
            'datetime': current_expected_dt,
            'reading': reading if reading is not None else "NC" # Use "NC" if no reading or parsed as None
        })
        current_expected_dt += datetime.timedelta(minutes=10)

    # Sort again just in case, though the loop should keep it sorted
    records_with_readings.sort(key=lambda x: x['datetime'])

    # Calculate 10-min differences and hourly accumulation
    # For daily summary, we mainly care about 'Daily Accumulation', but these intermediate
    # steps are part of the original logic you provided to derive it.
    
    # Stage 3 & 4: Calculate 10-min differences and Hourly Accumulation/Determinant
    # This part needs to ensure previous_numeric_reading correctly handles "NC"
    
    # Initialize previous_numeric_reading
    previous_numeric_reading = None
    for rec in records_with_readings:
        if rec['reading'] != "NC":
            previous_numeric_reading = rec['reading']
            break # Found the first valid reading

    if previous_numeric_reading is None and records_with_readings:
        # All readings are 'NC' or None, handle this case
        print(f"Warning: No numeric readings found in {filename} to calculate differences.")


    # Now iterate to calculate 10-min diffs and hourly accumulation
    hourly_diffs_window = collections.deque(maxlen=6) # Use deque for efficient windowing

    for i, record in enumerate(records_with_readings):
        current_reading_val = record['reading']
        ten_min_diff = "NC"

        if current_reading_val != "NC":
            if previous_numeric_reading is not None and i > 0: # Only calculate diff if there's a previous numeric reading
                ten_min_diff = current_reading_val - previous_numeric_reading
            else: # First valid reading, or first reading overall if no previous numeric
                ten_min_diff = 0
            previous_numeric_reading = current_reading_val # Update previous numeric reading
        
        # Add ten_min_diff to the record (even if "NC") for subsequent calculations
        record['ten_min_diff'] = ten_min_diff
        
        # Hourly accumulation logic
        hourly_diffs_window.append(ten_min_diff)

        # Check if it's the end of an hour (every 6th 10-min interval, starting from 00:00)
        # Or specifically, when the minute is 00 (e.g., 01:00, 02:00, etc.)
        if record['datetime'].minute == 0 and record['datetime'] != output_start_dt: # Not the very first record unless it's 00:00
            current_hourly_accumulation = 0
            has_nc_in_window = False
            
            # Sum the last 6 (including current 0-min diff) to get hourly total
            # We need to look back 6 entries including the current one.
            temp_window_for_hourly_calc = []
            for j in range(6):
                if i - j >= 0:
                    temp_window_for_hourly_calc.insert(0, records_with_readings[i-j]['ten_min_diff'])
                else: # Pad with 0 for records before start_dt in the first hour
                    temp_window_for_hourly_calc.insert(0, 0) 
            
            for diff in temp_window_for_hourly_calc:
                if diff == "NC":
                    has_nc_in_window = True
                    break
                else:
                    current_hourly_accumulation += diff
            
            if has_nc_in_window:
                record['hourly_accumulation'] = "NC"
                record['accumulation_determinant'] = "NC"
            else:
                record['hourly_accumulation'] = current_hourly_accumulation
                if current_hourly_accumulation == 0:
                    record['accumulation_determinant'] = "No Accumulation"
                elif current_hourly_accumulation > 0:
                    record['accumulation_determinant'] = "Accumulation"
                else:
                    record['accumulation_determinant'] = "Decline"
        else:
             # These are intermediate records, no hourly summary for them
            record['hourly_accumulation'] = ""
            record['accumulation_determinant'] = ""

    # Stage 5: Calculate Daily Accumulation (00:10 to 00:00 next day)
    daily_sums_to_assign = collections.defaultdict(lambda: {'sum': 0.0, 'has_nc': False, 'last_record_dt': None})

    for record in records_with_readings:
        hourly_acc = record.get('hourly_accumulation') # Use .get to safely access
        record_dt = record['datetime']

        # Determine which "day" this record contributes to, ending at a 00:00 timestamp
        daily_period_end_dt = get_daily_accumulation_end_dt(record_dt)

        if hourly_acc != "" and hourly_acc is not None: # Only process actual hourly accumulation values
            if hourly_acc == "NC":
                daily_sums_to_assign[daily_period_end_dt]['has_nc'] = True
            elif isinstance(hourly_acc, (int, float)):
                if not daily_sums_to_assign[daily_period_end_dt]['has_nc']:
                    daily_sums_to_assign[daily_period_end_dt]['sum'] += hourly_acc
            # Store the latest timestamp for each daily period, to assign the daily total to it
            daily_sums_to_assign[daily_period_end_dt]['last_record_dt'] = record_dt
    
    # Prepare the final list of daily summary records
    final_daily_summaries = []

    # Iterate through the calculated daily sums
    for daily_end_dt, info in sorted(daily_sums_to_assign.items()):
        daily_accumulation_value = info['sum']
        has_nc = info['has_nc']
        
        # The prompt implies the daily summary is assigned to the date *of the period*.
        # So, if accumulation ends at 2024/01/01 00:00, that's the daily summary for 2023/12/31.
        # If the accumulation ends at 2024/01/02 00:00, that's the daily summary for 2024/01/01.
        summary_date = daily_end_dt.date() if (daily_end_dt.hour == 0 and daily_end_dt.minute == 0) else daily_end_dt.date() - datetime.timedelta(days=1)
        
        if has_nc:
            daily_acc_output = "NC"
            summary_rain_no_rain = "Incomplete (NC in hourly data)"
        else:
            daily_acc_output = round(daily_accumulation_value, 1) # Round to 1 decimal place
            summary_rain_no_rain = "Rain" if daily_accumulation_value > 0 else "No Rain"
            
        # Refined Summary String including value and unit
        if daily_acc_output != "NC":
            summary_rain_no_rain_formatted = f"{summary_rain_no_rain} ({daily_acc_output:.1f} mm)"
        else:
            summary_rain_no_rain_formatted = summary_rain_no_rain # Keep as "Incomplete (NC...)"


        final_daily_summaries.append({
            'Date': summary_date.strftime("%d/%m/%Y"),
            'Daily Accumulation': daily_acc_output,
            'Summary (Rain/No Rain)': summary_rain_no_rain_formatted
        })
    
    # Create a DataFrame for the final summary (optional, but convenient for text/Excel consistency)
    if not final_daily_summaries:
        print(f"No daily summaries could be generated for {filename}.")
        return None, None
        
    summary_df = pd.DataFrame(final_daily_summaries)

    # --- Generate Text File Content for the summary ---
    text_content = f"Daily Rainfall Summary for {filename}\n"
    text_content += "------------------------------------------------------------------------\n"
    text_content += "Date            Daily Accumulation                Summary (Rain/No Rain)\n"
    text_content += "-" * 70 + "\n"
    for _, row in summary_df.iterrows():
        # Ensure consistent formatting for Daily Accumulation (e.g., "1.5" instead of "1.500000")
        daily_acc_str = f"{row['Daily Accumulation']:.1f}" if isinstance(row['Daily Accumulation'], (int, float)) else str(row['Daily Accumulation'])
        text_content += f"{row['Date']:<15} {daily_acc_str:<25} {row['Summary (Rain/No Rain)']}\n"

    # --- Generate Excel File Content (Base64 Encoded) for the summary using openpyxl ---
    output_excel = io.BytesIO()
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Daily Rainfall Summary"

    headers = ["Date", "Daily Accumulation", "Summary (Rain/No Rain)"]
    sheet.append(headers)

    for _, row in summary_df.iterrows():
        # Ensure 'Daily Accumulation' is stored as a number in Excel if possible
        daily_acc_for_excel = row['Daily Accumulation']
        if isinstance(daily_acc_for_excel, str) and daily_acc_for_excel == "NC":
            daily_acc_for_excel = "NC" # Keep as string if it's "NC"
        
        sheet.append([
            row['Date'],
            daily_acc_for_excel,
            row['Summary (Rain/No Rain)']
        ])
            
    workbook.save(output_excel)
    excel_content_base64 = base64.b64encode(output_excel.getvalue()).decode('utf-8')

    return text_content, excel_content_base64

# --- HOW TO USE THIS CODE ON YOUR LOCAL MACHINE ---

# 1. Save this code:
#    Save the entire content of this immersive block as a Python file (e.g., `process_rainfall_summaries.py`)
#    on your computer.

# 2. Prepare your files:
#    Place all your rainfall data text files (e.g., Alanao_Exp.TXT, Bagamelon_Exp.TXT)
#    into the specified `base_path` folder on your computer.

# 3. Specify your base folder path:
#    Change the `base_path` variable below to the actual path of the folder
#    where your rainfall data text files are located.
#    Example: base_path = 'C:/Users/YourUser/Documents/RainfallData/' (Windows)
#    Example: base_path = '/Users/YourUser/Documents/RainfallData/' (macOS/Linux)
#    Note: Ensure the path ends with a backslash or forward slash.
base_path = r"D:\OJT\BRB\Fill_In_data\2025\Experiment\\" # <<<<<<< IMPORTANT: CHANGE THIS TO YOUR ACTUAL FOLDER PATH

# 4. List the files to process:
#    This list maps file base names to their explicit processing start times.
#    The script will automatically append "_Exp.TXT" to these base names to find the input files.
files_to_process = [
    ("Alanao", datetime.datetime(2019, 4, 25, 11, 10)),
    ("Bagamelon", datetime.datetime(2019, 4, 25, 14, 10)),
    ("Balongay", datetime.datetime(2019, 4, 26, 11, 40)),
    ("Bato", datetime.datetime(2020, 12, 17, 12, 10)),
    ("Buhi", datetime.datetime(2020, 12, 17, 10, 10)),
    ("Caguscos", datetime.datetime(2020, 12, 18, 9, 10)),
    ("Calzada", datetime.datetime(2020, 12, 18, 13, 10)),
    ("Camaligan", datetime.datetime(2019, 4, 26, 10, 30)),
    ("Malabog", datetime.datetime(2020, 12, 18, 12, 10)),
    ("Ocampo", datetime.datetime(2020, 12, 27, 10, 10)),
    ("Ombao", datetime.datetime(2024, 12, 5, 15, 10)),
    ("Sipocot", datetime.datetime(2019, 4, 25, 13, 10))
]

# 5. Define an output directory for the summaries:
output_summary_directory = os.path.join(base_path, "Daily_Summaries") # Creates a subfolder for summaries
os.makedirs(output_summary_directory, exist_ok=True) # Ensure the output directory exists

# 6. Process each file and save its summary:
print(f"Output summaries will be saved in: {output_summary_directory}\n")

for file_base_name, _ in files_to_process:
    # Construct the actual input filename with "_Exp.TXT"
    actual_input_filename = f"{file_base_name}_Corrected.TXT"
    print(f"\n--- Processing {actual_input_filename} ---")
    input_file_path = os.path.join(base_path, actual_input_filename)
    
    if not os.path.exists(input_file_path):
        print(f"Error: Input file '{input_file_path}' not found. Skipping.")
        continue

    try:
        with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as f:
            file_content = f.read()
    except Exception as e:
        print(f"Error reading content of '{input_file_path}': {e}. Skipping.")
        continue

    # Process the content to get the summary outputs, passing the full filename
    text_summary, excel_summary_base64 = process_single_file_for_daily_summary(file_content, actual_input_filename)

    if text_summary is None or excel_summary_base64 is None:
        print(f"Could not generate summary for {actual_input_filename}. See warnings above.")
        continue

    # Save the text summary file
    # Output file names will be "Alanao_Daily_Summary.txt"
    output_txt_file = os.path.join(output_summary_directory, f"{file_base_name}_Daily_Summary.txt")
    try:
        with open(output_txt_file, "w", encoding="utf-8") as f:
            f.write(text_summary)
        print(f"Text summary for {actual_input_filename} saved to: {output_txt_file}")
    except Exception as e:
        print(f"Error saving text summary for {actual_input_filename}: {e}")

    # Save the Excel summary file
    # Output file names will be "Alanao_Daily_Summary.xlsx"
    output_excel_file = os.path.join(output_summary_directory, f"{file_base_name}_Daily_Summary.xlsx")
    try:
        decoded_excel_bytes = base64.b64decode(excel_summary_base64)
        with open(output_excel_file, "wb") as f:
            f.write(decoded_excel_bytes)
        print(f"Excel summary for {actual_input_filename} saved to: {output_excel_file}")
    except Exception as e:
        print(f"Error saving Excel summary for {actual_input_filename}: {e}")

print("\n--- All file processing complete. ---")


`HOURLY SUMMARY`

This code snippet does the following:

* Gives us an excel file and text file of the generalized 'hourly' summary of the CORRECTED data for ease of use

In [ ]:
# HOURLY SUMMARY #

import datetime
import collections
import os
import pandas as pd # Using pandas for easier DataFrame manipulation
import io
import base64 # For encoding Excel content to be displayable in this environment
import openpyxl # For direct Excel file creation

def _detect_delimiter_and_format(lines_to_sample, potential_delimiters=None, potential_date_formats=None):
    """
    Analyzes sample lines to detect the most likely delimiter and date format.
    """
    if potential_delimiters is None:
        potential_delimiters = ['\t', ',']

    if potential_date_formats is None:
        # Common formats: DD/MM/YYYY, YYYY/MM/DD, with and without leading zeros
        potential_date_formats = ["%d/%m/%Y", "%Y/%m/%d", "%#d/%#m/%Y", "%Y/%#m/%#d"] # Added %#d/%#m/%Y for dates like 1/1/2024

    best_delimiter = None
    best_date_format_str = None
    max_successful_parses = -1

    for delim in potential_delimiters:
        for date_fmt in potential_date_formats:
            successful_parses = 0
            for line in lines_to_sample:
                line_parts = line.strip().split(delim)
                # We need at least Date, Time, Reading, 10-min. Reading is at index 2 (0-indexed)
                if len(line_parts) >= 4: # Checking for at least 4 parts (Date, Time, Reading, 10-min)
                    date_str = line_parts[0].strip()
                    time_str = line_parts[1].strip()
                    value_str = line_parts[2].strip() # Reading is at index 2

                    try:
                        # Attempt to parse combined date and time
                        datetime.datetime.strptime(f"{date_str} {time_str}", f"{date_fmt} %H:%M")
                        # Attempt to convert reading to float, allowing 'NC' for now
                        if value_str != 'NC':
                            float(value_str)
                        successful_parses += 1
                    except ValueError:
                        pass
                    except IndexError: # Not enough parts after splitting
                        pass
            
            if successful_parses > max_successful_parses:
                max_successful_parses = successful_parses
                best_delimiter = delim
                best_date_format_str = f"{date_fmt} %H:%M"

    if max_successful_parses == 0:
        raise ValueError("Could not detect suitable delimiter and date format from sample lines. Please ensure the file has data in expected formats.")

    return best_delimiter, best_date_format_str

def get_daily_accumulation_end_dt(record_dt):
    """
    Determines the 00:00 datetime that marks the end of the daily accumulation period
    to which the given record_dt contributes.
    For example, 2024/01/01 10:00 -> 2024/01/02 00:00
    2024/01/01 00:00 -> 2024/01/01 00:00 (this 00:00 is the end of the previous day's accumulation)
    """
    # This function is retained from the original code but is no longer directly used for hourly summaries.
    # It would be used if calculating daily totals from hourly ones, but for just hourly, it's not strictly necessary.
    if record_dt.hour == 0 and record_dt.minute == 0:
        return record_dt.replace(hour=0, minute=0, second=0, microsecond=0)
    
    next_day = record_dt.date() + datetime.timedelta(days=1)
    return datetime.datetime(next_day.year, next_day.month, next_day.day, 0, 0)


def process_single_file_for_hourly_summary(file_content, filename):
    """
    Reads a single file's content, processes it, and returns the hourly summary
    in the requested format (Date, Time, Hourly Accumulation, Summary (Rain/No Rain)).

    Args:
        file_content (str): The raw string content of the input file.
        filename (str): The name of the file (used for logging and output naming).

    Returns:
        tuple: A tuple containing:
               - str: The content for the text summary file.
               - str: The base64 encoded content for the Excel summary file.
               Returns (None, None) if processing fails.
    """

    actual_data_records = collections.OrderedDict()
    
    initial_header_line = None
    start_data_line_index = 0
    file_delimiter = None
    input_datetime_format = None

    lines = file_content.strip().split('\n')
    
    if not lines:
        print(f"Error: Empty content for file '{filename}'.")
        return None, None
    
    # Attempt to detect initial ID line (e.g., "630381")
    first_line_stripped = lines[0].strip()
    temp_parts_tab = first_line_stripped.split('\t')
    temp_parts_comma = first_line_stripped.split(',')

    is_header_tab = len(temp_parts_tab) >= 1 and temp_parts_tab[0].isdigit() and all(p == '' for p in temp_parts_tab[1:])
    is_header_comma = len(temp_parts_comma) >= 1 and temp_parts_comma[0].isdigit() and all(p == '' for p in temp_parts_comma[1:])
    
    if is_header_tab or is_header_comma:
        initial_header_line = lines[0]
        start_data_line_index = 1
        
    sample_lines = lines[start_data_line_index : min(start_data_line_index + 20, len(lines))]
    
    if not sample_lines:
        print(f"Error: No data lines found in the input content for '{filename}' for format detection.")
        return None, None

    try:
        file_delimiter, input_datetime_format = _detect_delimiter_and_format(sample_lines)
        print(f"Auto-detected Delimiter: {repr(file_delimiter)}, Date/Time Format: '{input_datetime_format}' for {filename}")
    except ValueError as e:
        print(f"Error during auto-detection for {filename}: {e}")
        print("Attempting to proceed with default settings (tab delimiter, DD/MM/YYYY HH:MM).")
        file_delimiter = '\t'
        input_datetime_format = "%d/%m/%Y %H:%M"

    for line_num, line in enumerate(lines[start_data_line_index:]):
        line_parts = line.strip().split(file_delimiter)
        
        # Expecting at least Date, Time, Reading, 10-min difference (index 0, 1, 2, 3)
        if len(line_parts) >= 4:
            try:
                current_datetime = datetime.datetime.strptime(f"{line_parts[0].strip()} {line_parts[1].strip()}", input_datetime_format)
                
                try:
                    # Reading value is at index 2 (Reading column)
                    reading_value_str = line_parts[2].strip()
                    reading_value = float(reading_value_str) if reading_value_str != 'NC' else None # Store None for 'NC'
                except ValueError:
                    reading_value = None # Explicitly set to None if parsing fails
                
                actual_data_records[current_datetime] = reading_value

            except ValueError as e:
                print(f"Warning: Skipping malformed line {line_num + start_data_line_index + 1} in {filename}: '{line.strip()}' - Error parsing date/time or value: {e}")
            except IndexError:
                print(f"Warning: Skipping incomplete line {line_num + start_data_line_index + 1} in {filename}: '{line.strip()}' - Not enough columns.")
        else:
            print(f"Warning: Skipping malformed line {line_num + start_data_line_index + 1} in {filename}: '{line.strip()}' - Does not have expected number of columns (Date, Time, Reading, 10-min).")

    all_parsed_datetimes = sorted(actual_data_records.keys())

    if not all_parsed_datetimes:
        print(f"No valid data found or processed from the input file '{filename}' to reconstruct.")
        return None, None

    # Determine start and end points for filling intervals
    first_data_dt_from_file = all_parsed_datetimes[0]
    last_data_dt_from_file = all_parsed_datetimes[-1]

    # Adjust start and end points to nearest 10-minute interval
    output_start_dt = first_data_dt_from_file.replace(minute=(first_data_dt_from_file.minute // 10) * 10, second=0, microsecond=0)
    if output_start_dt.minute % 10 != 0: # If it wasn't already on a 10-min mark, round up
        output_start_dt += datetime.timedelta(minutes=(10 - (output_start_dt.minute % 10)))

    output_end_dt = last_data_dt_from_file.replace(minute=(last_data_dt_from_file.minute // 10) * 10, second=0, microsecond=0)

    # Fill missing intervals
    current_expected_dt = output_start_dt
    records_with_readings = []
    
    while current_expected_dt <= output_end_dt:
        reading = actual_data_records.get(current_expected_dt)
        records_with_readings.append({
            'datetime': current_expected_dt,
            'reading': reading if reading is not None else "NC" # Use "NC" if no reading or parsed as None
        })
        current_expected_dt += datetime.timedelta(minutes=10)

    # Sort again just in case, though the loop should keep it sorted
    records_with_readings.sort(key=lambda x: x['datetime'])

    # Initialize previous_numeric_reading for 10-min diff calculations
    previous_numeric_reading = None
    for rec in records_with_readings:
        if rec['reading'] != "NC":
            previous_numeric_reading = rec['reading']
            break # Found the first valid reading

    if previous_numeric_reading is None and records_with_readings:
        print(f"Warning: No numeric readings found in {filename} to calculate differences.")

    # Calculate 10-min differences and Hourly Accumulation/Determinant
    for i, record in enumerate(records_with_readings):
        current_reading_val = record['reading']
        ten_min_diff = "NC"

        if current_reading_val != "NC":
            if previous_numeric_reading is not None and i > 0:
                ten_min_diff = current_reading_val - previous_numeric_reading
            else:
                ten_min_diff = 0
            previous_numeric_reading = current_reading_val
        
        record['ten_min_diff'] = ten_min_diff
        
        # Hourly accumulation logic: calculate at the top of the hour (e.g., 01:00, 02:00)
        # This record is the *end* of the hourly period (e.g., 01:00 covers 00:10 to 01:00)
        if record['datetime'].minute == 0:
            current_hourly_accumulation = 0
            has_nc_in_window = False
            
            # Look back 6 entries including the current one for the hourly window
            temp_window_for_hourly_calc = []
            for j in range(6):
                # Ensure we don't go out of bounds at the very beginning of the data
                if i - j >= 0:
                    temp_window_for_hourly_calc.insert(0, records_with_readings[i-j]['ten_min_diff'])
                else:
                    # If we don't have a full 6 records at the start, treat missing as 0
                    temp_window_for_hourly_calc.insert(0, 0)
            
            for diff in temp_window_for_hourly_calc:
                if diff == "NC":
                    has_nc_in_window = True
                    break
                else:
                    current_hourly_accumulation += diff
            
            if has_nc_in_window:
                record['hourly_accumulation'] = "NC"
                record['accumulation_determinant'] = "NC"
            else:
                record['hourly_accumulation'] = current_hourly_accumulation
                if current_hourly_accumulation == 0:
                    record['accumulation_determinant'] = "No Accumulation"
                elif current_hourly_accumulation > 0:
                    record['accumulation_determinant'] = "Accumulation"
                else:
                    record['accumulation_determinant'] = "Decline"
        else:
             # These are intermediate records, no hourly summary for them
            record['hourly_accumulation'] = ""
            record['accumulation_determinant'] = ""

    # Stage 5: Prepare Hourly Summary Records
    final_hourly_summaries = []

    for record in records_with_readings:
        # We only want records that represent the end of an hour's accumulation (e.g., 01:00, 02:00 etc.)
        if record['datetime'].minute == 0:
            hourly_acc_value = record.get('hourly_accumulation')
            
            if hourly_acc_value is not None and hourly_acc_value != "":
                summary_date = record['datetime'].strftime("%d/%m/%Y")
                summary_time = record['datetime'].strftime("%H:%M") # Format as HH:MM
                
                if hourly_acc_value == "NC":
                    hourly_acc_output = "NC"
                    summary_rain_no_rain = "Incomplete (NC in 10-min data)"
                else:
                    hourly_acc_output = round(hourly_acc_value, 1) # Round to 1 decimal place
                    summary_rain_no_rain = "Rain" if hourly_acc_value > 0 else "No Rain"
                
                # Refined Summary String including value and unit
                if hourly_acc_output != "NC":
                    summary_rain_no_rain_formatted = f"{summary_rain_no_rain} ({hourly_acc_output:.1f} mm)"
                else:
                    summary_rain_no_rain_formatted = summary_rain_no_rain


                final_hourly_summaries.append({
                    'Date': summary_date,
                    'Time': summary_time,
                    'Hourly Accumulation': hourly_acc_output,
                    'Summary (Rain/No Rain)': summary_rain_no_rain_formatted
                })

    if not final_hourly_summaries:
        print(f"No hourly summaries could be generated for {filename}.")
        return None, None
        
    # Convert to DataFrame for easier handling and sorting
    summary_df = pd.DataFrame(final_hourly_summaries)
    # Ensure sorting by Date and then Time by converting back to datetime objects temporarily
    summary_df['TempDateTime'] = pd.to_datetime(summary_df['Date'] + ' ' + summary_df['Time'], format='%d/%m/%Y %H:%M')
    summary_df = summary_df.sort_values(by='TempDateTime').drop(columns='TempDateTime')


    # --- Generate Text File Content for the summary ---
    text_content = f"Hourly Rainfall Summary for {filename}\n"
    text_content += "------------------------------------------------------------------------\n"
    text_content += "Date            Time    Hourly Accumulation       Summary (Rain/No Rain)\n"
    text_content += "-" * 70 + "\n"
    for _, row in summary_df.iterrows():
        # Ensure consistent formatting for Hourly Accumulation
        hourly_acc_str = f"{row['Hourly Accumulation']:.1f}" if isinstance(row['Hourly Accumulation'], (int, float)) else str(row['Hourly Accumulation'])
        text_content += f"{row['Date']:<15} {row['Time']:<7} {hourly_acc_str:<25} {row['Summary (Rain/No Rain)']}\n"

    # --- Generate Excel File Content (Base64 Encoded) for the summary using openpyxl ---
    output_excel = io.BytesIO()
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Hourly Rainfall Summary"

    headers = ["Date", "Time", "Hourly Accumulation", "Summary (Rain/No Rain)"]
    sheet.append(headers)

    for _, row in summary_df.iterrows():
        # Ensure 'Hourly Accumulation' is stored as a number in Excel if possible
        hourly_acc_for_excel = row['Hourly Accumulation']
        if isinstance(hourly_acc_for_excel, str) and hourly_acc_for_excel == "NC":
            hourly_acc_for_excel = "NC" # Keep as string if it's "NC"
        
        sheet.append([
            row['Date'],
            row['Time'],
            hourly_acc_for_excel,
            row['Summary (Rain/No Rain)']
        ])
            
    workbook.save(output_excel)
    excel_content_base64 = base64.b64encode(output_excel.getvalue()).decode('utf-8')

    return text_content, excel_content_base64

# --- HOW TO USE THIS CODE ON YOUR LOCAL MACHINE ---

# 1. Save this code:
#    Save the entire content of this immersive block as a Python file (e.g., `process_rainfall_summaries.py`)
#    on your computer.

# 2. Prepare your files:
#    Place all your rainfall data text files (e.g., Alanao_Exp.TXT, Bagamelon_Exp.TXT)
#    into the specified `base_path` folder on your computer.

# 3. Specify your base folder path:
#    Change the `base_path` variable below to the actual path of the folder
#    where your rainfall data text files are located.
#    Example: base_path = 'C:/Users/YourUser/Documents/RainfallData/' (Windows)
#    Example: base_path = '/Users/YourUser/Documents/RainfallData/' (macOS/Linux)
#    Note: Ensure the path ends with a backslash or forward slash.
base_path = r"D:\OJT\BRB\Fill_In_data\2025\Experiment\\" # <<<<<<< IMPORTANT: CHANGE THIS TO YOUR ACTUAL FOLDER PATH

# 4. List the files to process:
#    This list maps file base names to their explicit processing start times.
#    The script will automatically append "_Exp.TXT" to these base names to find the input files.
files_to_process = [
    ("Alanao", datetime.datetime(2019, 4, 25, 11, 10)),
    ("Bagamelon", datetime.datetime(2019, 4, 25, 14, 10)),
    ("Balongay", datetime.datetime(2019, 4, 26, 11, 40)),
    ("Bato", datetime.datetime(2020, 12, 17, 12, 10)),
    ("Buhi", datetime.datetime(2020, 12, 17, 10, 10)),
    ("Caguscos", datetime.datetime(2020, 12, 18, 9, 10)),
    ("Calzada", datetime.datetime(2020, 12, 18, 13, 10)),
    ("Camaligan", datetime.datetime(2019, 4, 26, 10, 30)),
    ("Malabog", datetime.datetime(2020, 12, 18, 12, 10)),
    ("Ocampo", datetime.datetime(2020, 12, 27, 10, 10)),
    ("Ombao", datetime.datetime(2024, 12, 5, 15, 10)),
    ("Sipocot", datetime.datetime(2019, 4, 25, 13, 10))
]

# 5. Define an output directory for the summaries:
output_summary_directory = os.path.join(base_path, "Hourly_Summaries") # Creates a subfolder for summaries
os.makedirs(output_summary_directory, exist_ok=True) # Ensure the output directory exists

# 6. Process each file and save its summary:
print(f"Output summaries will be saved in: {output_summary_directory}\n")

for file_base_name, _ in files_to_process:
    # Construct the actual input filename with "_Exp.TXT"
    actual_input_filename = f"{file_base_name}_Corrected.TXT"
    print(f"\n--- Processing {actual_input_filename} ---")
    input_file_path = os.path.join(base_path, actual_input_filename)
    
    if not os.path.exists(input_file_path):
        print(f"Error: Input file '{input_file_path}' not found. Skipping.")
        continue

    try:
        with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as f:
            file_content = f.read()
    except Exception as e:
        print(f"Error reading content of '{input_file_path}': {e}. Skipping.")
        continue

    # Process the content to get the summary outputs, passing the full filename
    text_summary, excel_summary_base64 = process_single_file_for_hourly_summary(file_content, actual_input_filename)

    if text_summary is None or excel_summary_base64 is None:
        print(f"Could not generate summary for {actual_input_filename}. See warnings above.")
        continue

    # Save the text summary file
    # Output file names will be "Alanao_Hourly_Summary.txt"
    output_txt_file = os.path.join(output_summary_directory, f"{file_base_name}_Hourly_Summary.txt")
    try:
        with open(output_txt_file, "w", encoding="utf-8") as f:
            f.write(text_summary)
        print(f"Text summary for {actual_input_filename} saved to: {output_txt_file}")
    except Exception as e:
        print(f"Error saving text summary for {actual_input_filename}: {e}")

    # Save the Excel summary file
    # Output file names will be "Alanao_Hourly_Summary.xlsx"
    output_excel_file = os.path.join(output_summary_directory, f"{file_base_name}_Hourly_Summary.xlsx")
    try:
        decoded_excel_bytes = base64.b64decode(excel_summary_base64)
        with open(output_excel_file, "wb") as f:
            f.write(decoded_excel_bytes)
        print(f"Excel summary for {actual_input_filename} saved to: {output_excel_file}")
    except Exception as e:
        print(f"Error saving Excel summary for {actual_input_filename}: {e}")

print("\n--- All file processing complete. ---")


Prepared by: Carl Benjamin L. Buenaflor | Bicol University | Meteo_ID2022
